In [2]:
import ipynb
import geopandas as gpd
import shapely.geometry as geom
from shapely.geometry import mapping
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [3]:
def build_baseline_mining_mask(
    baseline_cube: xr.Dataset,
    ndvi_bare_max: float = 0.25,
    ndvi_var_max: float = 0.02,
    bsi_min: float = 0.0,
    visualize: bool = False,
    mine_boundary=None,
):
    """
    STEP B0 — Baseline Mining Mask Construction

    Parameters
    ----------
    baseline_cube : xr.Dataset
        Must contain NDVI, BSI, B11, B12 with dims (time, y, x)

    ndvi_bare_max : float
        Maximum NDVI allowed for excavation

    ndvi_var_max : float
        Maximum NDVI temporal variance (stability constraint)

    bsi_min : float
        Minimum BSI value for bare soil

    visualize : bool
        If True, plots baseline mining mask

    mine_boundary : GeoDataFrame (optional)
        Boundary overlay for sanity check

    Returns
    -------
    baseline_mining_mask : xr.DataArray
        Binary excavation initialization mask (0/1)

    baseline_stats : xr.Dataset
        Per-pixel baseline statistics (for audit / figures)
    """

    # ---------- GROUP A: Baseline statistics ----------
    baseline_stats = xr.Dataset({
        "ndvi_median": baseline_cube["NDVI"].median("time"),
        "ndvi_var": baseline_cube["NDVI"].var("time"),
        "bsi_median": baseline_cube["BSI"].median("time"),
        "swir1_median": baseline_cube["B11"].median("time"),
        "swir2_median": baseline_cube["B12"].median("time"),
    })

    # ---------- GROUP B + C: Physical constraints ----------
    baseline_mining_mask = (
        (baseline_stats["ndvi_median"] < ndvi_bare_max) &
        (baseline_stats["ndvi_var"] < ndvi_var_max) &
        (baseline_stats["bsi_median"] > bsi_min)
    ).astype(np.uint8)

    baseline_mining_mask.name = "BaselineMiningMask"

    # ---------- GROUP D: Visual sanity check ----------
    if visualize:
        plt.figure(figsize=(6, 6))
        baseline_mining_mask.plot(cmap="Reds")
        if mine_boundary is not None:
            mine_boundary.boundary.plot(
                ax=plt.gca(), color="cyan", linewidth=1
            )
        plt.title("Baseline Mining Mask (Pre-existing Excavation)")
        plt.axis("off")
        plt.show()

    return baseline_mining_mask, baseline_stats


In [4]:
def merge_baseline_with_monitoring(
    baseline_mining_mask: xr.DataArray,
    excavation_maps: dict,
):
    """
    STEP B1 — Merge Baseline Excavation with New Excavation

    E_final(p,t) = BaselineMiningMask(p) OR NewExcavationMask(p,t)

    Parameters
    ----------
    baseline_mining_mask : xr.DataArray
        Output from build_baseline_mining_mask

    excavation_maps : dict
        {time: xr.DataArray} anomaly-derived excavation masks

    Returns
    -------
    final_excavation_maps : dict
        {time: xr.DataArray} merged excavation maps
    """

    final_excavation_maps = {}

    for t, exc_map in excavation_maps.items():
        final_excavation_maps[t] = (
            baseline_mining_mask | exc_map
        ).astype(np.uint8)

    return final_excavation_maps
